In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib.patches import Circle
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

from astropy.constants import G
from astropy.table import Table
import astropy.units as u
import astropy.coordinates as coord
from astropy.io import ascii

from pyia import GaiaData
import gala.coordinates as gc

In [ ]:
g = GaiaData('/Users/adrian/projects/gd1-dr2/data/gd1-master.fits')

In [ ]:
stream = g[g.pm_mask & g.gi_cmd_mask & (g.phi2>-5*u.deg)]

In [ ]:
fig = plt.figure(figsize=(8, 6))

gs = GridSpec(nrows=4, ncols=2)
axes = [fig.add_subplot(gs[:2, 0]), 
        fig.add_subplot(gs[:2, 1]), 
        fig.add_subplot(gs[2:, :])]

# fig, ax = plt.subplots(1, 1, figsize=(9, 4), 
#                        constrained_layout=True)

# ------------------------------------------
# Lower panel
#
ax = axes[2]
ax.plot(stream.phi1, stream.phi2, 
        color='k', marker='o', ms=2.5, 
        alpha=0.7, mec='none', ls='none')

h = 2.75
r = 1.5
kw = dict(radius=r, alpha=0.8, edgecolor='tab:red', 
          facecolor='none', linewidth=2)

y0 = 0.
y1 = 0.4
xs = np.arange(-47, -13+1e-3, h)

field_cens = []
for x in xs:
    
    if x < -38 or x > -28:
        circ = Circle((x, y0), **kw)
        ax.add_patch(circ)
        field_cens.append([x, y0])
    
    else:
        circ = Circle((x, y1), **kw)
        ax.add_patch(circ)
        field_cens.append([x, y1])
    
n_fields = len(field_cens)
# ax.set_title('GD-1: {0} fields'.format(n_fields))

ax.set_xlabel('$\phi_1$ [deg]')
ax.set_ylabel('$\phi_2$ [deg]')

ax.set_xlim(-55, -5)
ax.set_ylim(-5, 5)

ax.set_aspect('equal')


# ------------------------------------------
# Upper left panel
#
ax = axes[0]

ax.set_xlim(-8, 8)
ax.set_ylim(0, 4)

ax.set_xlabel(r'$\psi$ [deg]')
ax.set_ylabel(r'$\rho / \rho_0$')

# ------------------------------------------
# Upper right panel
#
ax = axes[1]

binc, n_bg = np.load('g_nbg.npy')
binc, n_stream = np.load('g_nstream.npy')

ax.plot(binc, np.cumsum(n_stream) / np.sqrt(np.cumsum(n_bg)) - sn_poly(binc), 
        marker='', lw=3., color='k')

ax.set_xlabel(r'10$\sigma$ limiting $g$ [mag]')
ax.set_ylabel('cumulative S/N\n per HSC field')

ax.set_xticks(np.arange(20, 26+1e-3, 1))
ax.set_xlim(20, 26)

ax.set_ylim(0, 35)

# ax.axvline(23., alpha=0.4, 
#            color='k', linestyle='--', zorder=-10)
ax.axvline(25.5, alpha=0.6, 
           color='k', zorder=-10)

# -----

fig.set_facecolor('w')
fig.tight_layout()

fig.savefig('../figure1.pdf')

In [ ]:
g_time = 500*u.second
i_time = 500*u.second
(n_fields * (g_time + i_time)).to(u.hour)

---

In [ ]:
from scipy.stats import poisson

In [ ]:
N1 = 1000.
Nbg1 = 400.

N2 = 1091.
Nbg2 = 410.

(N1 - Nbg1) / (N2 - Nbg2)

In [ ]:
frac = ((poisson.rvs(N1, size=4096) - poisson.rvs(Nbg1, size=4096)) /
        (poisson.rvs(N2, size=4096) - poisson.rvs(Nbg2, size=4096)))
plt.hist(frac, bins=np.linspace(0.5, 1.5, 32));
np.std(frac)

In [ ]:
a = N1 - Nbg1
erra = np.sqrt(N1 + Nbg1)

b = N2 - Nbg2
errb = np.sqrt(N2 + Nbg2)

In [ ]:
(a/b) * np.sqrt(erra**2/a**2 + errb**2/b**2)